In [1]:
!which python
!which uv
!pwd

/home/user/.cache/uv/archive-v0/WFBFj9Evhjjky68nZNXvH/bin/python
/usr/bin/uv
/home/user/unshared/finetuning-notebooks


In [2]:
!uv pip install unsloth

Audited 1 package in 19ms


In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.7.2: Fast Llama patching. Transformers: 4.53.1.
   \\   /|    NVIDIA GeForce RTX 3060. Num GPUs = 1. Max memory: 11.631 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.6. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.7.2 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [5]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("mlabonne/FineTome-100k", split = "train")

In [ ]:
# len(dataset)
dataset[22]

{'conversations': [{'from': 'human',
   'value': 'Can you explain how this C++ program checks if a number is even or odd?'},
  {'from': 'gpt',
   'value': 'Certainly! This C++ program uses a function called "is_even" to check whether a given number is even or odd. Here\'s a breakdown of the code:\n\n1. First, we include the necessary library, iostream, to enable input and output operations.\n\n2. The "is_even" function is defined with a parameter "num" of type int. This function returns a boolean value, True if the number is even, and False if it is odd.\n\n3. Inside the "is_even" function, we use the modulo operator (%) to check if "num" is divisible evenly by 2. If the remainder is 0, it means the number is even, so we return True. Otherwise, if the remainder is not 0, the number is odd, and we return False.\n\n4. In the "main" function, we declare an integer variable called "num" to store the user input.\n\n5. We prompt the user to enter a number using the cout statement and the << 

In [7]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [8]:
dataset[22]

{'conversations': [{'content': 'Can you explain how this C++ program checks if a number is even or odd?',
   'role': 'user'},
  {'content': 'Certainly! This C++ program uses a function called "is_even" to check whether a given number is even or odd. Here\'s a breakdown of the code:\n\n1. First, we include the necessary library, iostream, to enable input and output operations.\n\n2. The "is_even" function is defined with a parameter "num" of type int. This function returns a boolean value, True if the number is even, and False if it is odd.\n\n3. Inside the "is_even" function, we use the modulo operator (%) to check if "num" is divisible evenly by 2. If the remainder is 0, it means the number is even, so we return True. Otherwise, if the remainder is not 0, the number is odd, and we return False.\n\n4. In the "main" function, we declare an integer variable called "num" to store the user input.\n\n5. We prompt the user to enter a number using the cout statement and the << operator.\n\n6.

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [10]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

In [11]:
tokenizer.decode(trainer.train_dataset[22]["input_ids"])

'<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nCan you explain how this C++ program checks if a number is even or odd?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nCertainly! This C++ program uses a function called "is_even" to check whether a given number is even or odd. Here\'s a breakdown of the code:\n\n1. First, we include the necessary library, iostream, to enable input and output operations.\n\n2. The "is_even" function is defined with a parameter "num" of type int. This function returns a boolean value, True if the number is even, and False if it is odd.\n\n3. Inside the "is_even" function, we use the modulo operator (%) to check if "num" is divisible evenly by 2. If the remainder is 0, it means the number is even, so we return True. Otherwise, if the remainder is not 0, the number is odd, and we return False.\n\

In [12]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[22]["labels"]])

'                                                     Certainly! This C++ program uses a function called "is_even" to check whether a given number is even or odd. Here\'s a breakdown of the code:\n\n1. First, we include the necessary library, iostream, to enable input and output operations.\n\n2. The "is_even" function is defined with a parameter "num" of type int. This function returns a boolean value, True if the number is even, and False if it is odd.\n\n3. Inside the "is_even" function, we use the modulo operator (%) to check if "num" is divisible evenly by 2. If the remainder is 0, it means the number is even, so we return True. Otherwise, if the remainder is not 0, the number is odd, and we return False.\n\n4. In the "main" function, we declare an integer variable called "num" to store the user input.\n\n5. We prompt the user to enter a number using the cout statement and the << operator.\n\n6. The user input is then read and stored in the "num" variable using the cin statement a

In [13]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3060. Max memory = 11.631 GB.
3.441 GB of memory reserved.


In [14]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,0.779200
2,0.814300
3,1.080800
4,0.892900
5,0.772800
6,0.942800
7,0.606600
8,1.003100
9,0.880200
10,0.760900


In [15]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

347.22 seconds used for training.
5.79 minutes used for training.
Peak reserved memory = 3.441 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 29.585 %.
Peak reserved memory for training % of max memory = 0.0 %.


In [16]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nContinue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThe Fibonacci sequence is a series of numbers in which each number is the sum of the two preceding numbers. The sequence you provided starts with 1, 1, 2, 3, 5, and 8. Here are the next three numbers in the sequence:\n9, 14, 23,']

In [17]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

1, 13, 21, 34, 55, 89.<|eot_id|>


In [18]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/chat_template.jinja',
 'lora_model/tokenizer.json')

In [ ]:
## if true and run, the modelfile cannot be generated from the tokenizer later
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Describe a tall tower in the capital of France."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

The Eiffel Tower, located in Paris, is an iconic 324-meter-tall iron lattice structure. It was designed and built in 1889 to serve as the entrance arch for the World's Fair. The tower has become a symbol of Paris and a famous tourist destination.<|eot_id|>


In [20]:
model.save_pretrained_gguf("model", tokenizer)

Unsloth: You have 2 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 14.02 out of 23.22 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [00:00<00:00, 38.08it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model/pytorch_model-00001-of-00002.bin...
Unsloth: Saving model/pytorch_model-00002-of-00002.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at model into q8_0 GGUF format.
The output location will be /home/user/unshared/finetuning-notebooks/model/unsloth.Q8_0.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: loading model part 'pytorch_mode

In [21]:
model.save_pretrained_gguf("model_f16", tokenizer, quantization_method = "f16")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 13.72 out of 23.22 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [00:00<00:00, 41.39it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model_f16/pytorch_model-00001-of-00002.bin...
Unsloth: Saving model_f16/pytorch_model-00002-of-00002.bin...
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['f16'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at model_f16 into f16 GGUF format.
The output location will be /home/user/unshared/finetuning-notebooks/model_f16/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model_f16
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,          

In [22]:
model.save_pretrained_gguf("model_q4_k_m", tokenizer, quantization_method = "q4_k_m")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 13.84 out of 23.22 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [00:00<00:00, 41.61it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model_q4_k_m/pytorch_model-00001-of-00002.bin...
Unsloth: Saving model_q4_k_m/pytorch_model-00002-of-00002.bin...
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at model_q4_k_m into bf16 GGUF format.
The output location will be /home/user/unshared/finetuning-notebooks/model_q4_k_m/unsloth.BF16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model_q4_k_m
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_fre

In [23]:
print(tokenizer._ollama_modelfile)


FROM {__FILE_LOCATION__}
TEMPLATE """{{ if .Messages }}
{{- if or .System .Tools }}<|start_header_id|>system<|end_header_id|>
{{- if .System }}

{{ .System }}
{{- end }}
{{- if .Tools }}

You are a helpful assistant with tool calling capabilities. When you receive a tool call response, use the output to format an answer to the original use question.
{{- end }}
{{- end }}<|eot_id|>
{{- range $i, $_ := .Messages }}
{{- $last := eq (len (slice $.Messages $i)) 1 }}
{{- if eq .Role "user" }}<|start_header_id|>user<|end_header_id|>
{{- if and $.Tools $last }}

Given the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.

Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}. Do not use variables.

{{ $.Tools }}
{{- end }}

{{ .Content }}<|eot_id|>{{ if $last }}<|start_header_id|>assistant<|end_header_id|>

{{ end }}
{{- else if eq .Role "assistant" }}<|start_header

In [24]:
## ollama serve
!ollama create unsloth_model -f ./model/Modelfile
# !ollama create unsloth_model -f ./model_f16/Modelfile
# !ollama create unsloth_model -f ./model_q4_k_m/Modelfile

gathering model components ⠋ gathering model components ⠙ gathering model components ⠸ gathering model components ⠼ gathering model components ⠼ gathering model components ⠴ gathering model components ⠦ gathering model components ⠇ gathering model components ⠇ gathering model components ⠏ gathering model components ⠋ gathering model components ⠹ gathering model components ⠹ gathering model components ⠸ gathering model components ⠼ gathering model components ⠴ gathering model components ⠧ gathering model components ⠧ gathering model components ⠇ gathering model components ⠋ gathering model components ⠋ gathering model components ⠙ gathering model components ⠹ gathering model components ⠼ gathering model components ⠼ gathering model components ⠴ gathering model components ⠦ gathering model components ⠧ gathering model components ⠏ gathering model components ⠏ gathering model components ⠋ gathering model components ⠙ gathering model components ⠹ gathering model components ⠸ gathering mode